# Лабораторная работа №7. Решающие деревья. Усиление распознавателей

В данной лабораторной работе рассматривается использование бинарных распознающих деревьев и их ансамблей для решения задачи классификации.

Распознающим бинарным деревом называется дерево, с каждой вершиной $v$ которого связаны:

1. Некоторое подмножество $\mathcal{X}_v \subset \mathcal{X}$;
1. Подвыборка $\mathcal{D}_v \subset \mathcal{D}$ обучающей выборки $\mathcal{D}$, такая, что
$$
    \mathcal{D}_v = \left\{(\mathbf{x}, t) \in \mathcal{D} \,\vert\, \mathbf{x} \in \mathcal{X}_v\right\};
$$
1. Некоторое правило $f_v: \mathcal{X} \to \{0, 1\}$, определяющее разбиение множества $\mathcal{X}$ на $K$ непересекающихся подмножеств.

Цель построения дерева решений состоит либо в классификации векторов $\mathbf{x}$, либо в оценке математического ожидания отклика при данном значении $\mathbf{x}$.
Процесс принятия решения начинается с корневой вершины и состоит в последовательном применении правил, связанных с вершинами дерева.
Результатом этого процесса является определение терминальной вершины $v$, такой что $\mathbf{x} \in \mathcal{X}_v$.

Распознающие деревья редко применяют в качестве самостоятельного распознавателя, так как хорошего качества распознавания от них редко удается добиться.
Вместо этого их широко применяют в качестве «сырья» в методах усиления распознавателей, где синтез решения отдельных классификаторов, составляющих ансамбль, осуществляется путем их голосования.

Метод баггинга основан на формировании обучающей выборки для каждого классификатора ансамбля с помощью бутстрепа, то есть случайной выборки с возвращениями из исходной обучающей выборки.

Суть метода случайных лесов заключается в том, что для каждого дерева ансамбля на стадии расщепления вершин используется только некоторое подмножество случайно отбираемых признаков.
Чаще всего размерность такого подмножества выбирается близкой к $\sqrt{p}$, где $p$ $-$ размерность всего пространства признаков.

Идея метода бустинга состоит в том, что классификаторы ансамбля строятся последовательно, и на каждой итерации происходит коррекция (перевзвешивание) образов обучающей выборки.
Коррекция осуществляется таким образом, чтобы соответствующий классификатор дал меньше ошибок на тех образах, на которых часто делали ошибки классификаторы, построенные на предыдущих итерациях алгоритма.
Кроме того, каждому классификатору приписывается некоторый вес исходя из количества допущенных им ошибок.

In [ ]:
import numpy as np
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection
import sklearn.tree
import sklearn.ensemble
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['axes.grid'] = True
plt.rcParams['figure.constrained_layout.use'] = True

## Задание №1

Сгенерируйте датасет с 2 признаками размеро 3000 строк (`n_informative=2`, остальные параметры следует установить в 0).
В качестве `random_state` используйте свой порядковый номер в списке.
Разделите датасет на обучающую и тестовую выборки.
Размер тестовой выборки следует выбрать равным 20% от размера всей выборки.
Не забывайте про параметр `random_state` для повторяемости результатов.

In [ ]:
x, y = sk.datasets.make_classification(???)

_, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*5, 4))
ax0.scatter
ax0.set(??? 'Обучающая выборка')

## Задание №2

Найдите наилучшие параметры решающего дерева `DecisionTreeClassifier`.
Для этого постройте графики зависимостей _accuracy_ от сложности модели (максимального числа разделений) для обучающей и тестовой выборок.
После чего выберите оптимальное значение сложности модели.

**Замечание.**
Оптимальные параметры следует выбирать по наилучшему значению на тестовой выборке.
Если в исследуемой зависимости наблюдается несколько пиков с наилучшим значением, то стоит принять в рассмотрение также результат на обучающей выборке.

In [ ]:
max_depth = range(1, 30)
criterions = [???]
_, axes = plt.subplots(1, len(criterions), figsize=(len(criterions)*5, 4))
for criterion, ax in zip(criterions, axes):
    metrics_train, metrics_test = [], []
    ???

    ax.plot(max_depth, metrics_train, label='train')
    ax.plot(???)
    ax.set(xlabel='max depth', ylabel='accuracy', title='Точность (accuracy) для критерия ' + criterion)
    ax.legend()
    
    print(criterion, ???, 'max_depth:', ???)

**Вопросы:**

1. Какие критерии были рассмотрены?
1. Запишите полученную наилучшую точность и максимальное число разделений для каждого критерия.
1. Какой критерий разделения дает наилучший результат?
1. Наблюдается ли эффект переобучения (если да, то с чем он связан)?

**Ответы:**

1. ...
1. ...

Визуализируйте разделение пространства признаков для наилучшей модели.

In [ ]:
x_min, x_max = x_test[:, 0].min() - 1, x_test[:, 0].max() + 1
y_min, y_max = x_test[:, 1].min() - 1, x_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

best_model = ???
Z = best_model.predict(np.c_[xx.ravel(), yy.ravel(), np.zeros((xx.ravel().shape[0], x_test.shape[1] - 2))])
plt.contourf(xx, yy, Z.reshape(xx.shape), cmap=plt.cm.Paired, alpha=0.5)
plt.scatter(???)

**Вопросы:**

1. Чем характерно отличается полученное разбиение от тех вариантов, что получались в прошлых лабораторных работах?

**Ответы:**

1. ...

## Задание №3

Найдите наилучшие параметры ансамбля, полученного баггингом `BaggingClassifier`.
Для этого воспользуйтесь функцией `GridSearchCV` с указанием кросс-валидации.
Рассмотрите такие параметры перебора:

* Максимальное число разделений (от 1 до 20)
* Максимальное доля обучающей выборки
* Количество деревьев в ансамбле (от 1 до 100)

Шаг для перебора каждого из параметров можете выбрать самостоятельно.
Визуализируйте разделение пространства признаков для наилучшей модели.

In [ ]:
%%time

param_grid = {
    'base_estimator__max_depth' : ???,
    'max_samples' : ??? np.linspace,
    'n_estimators' : ??? np.linspace.astype(int)}

gs = sk.model_selection.GridSearchCV(???,
                                     param_grid=param_grid, return_train_score=True, n_jobs=-1)
???

Z = gs.predict(???)
???

print(gs.best_params_)
print('train', gs.score(x_train, y_train))
print('test', ???)

**Вопросы:**

1. Удалось ли увеличить качество модели по сравнению с одним распознающим деревом?
1. Как и почему изменилась форма разделяющей поверхности?
1. Как изменилась оптимальная сложность деревьев по сравнению с предыдущим заданием и с чем это связано?

**Ответы:**

1. ...
1. ...

Проанализируйте зависимости _accuracy_ от конкретных параметров.
Для этого используйте информацию из словаря `cv_results_`, беря во внимание полученные оптимальные параметры `best_params_`.
Постройте графики зависимости среднего значения _accuracy_ для каждого из параметров, фиксируя остальные параметры в значениях, взятых из `best_params_`.
Графики должны быть построены для обучающей и тестовой выборок, также показан разброс на тестовой выбоке (разброс вычисляется благодаря кросс-валидации).

In [ ]:
print(gs.best_params_)

_, axes = plt.subplots(1, len(gs.best_params_), figsize=(len(gs.best_params_)*5, 4))
for param, ax in zip(gs.best_params_.keys(), axes):
    reduced_dict = {k:v for k, v in gs.best_params_.items() if k != param}
    idx = [i for i, x in enumerate(gs.cv_results_['params']) if reduced_dict.items() <= x.items()]

    x, y, t, s = (gs.cv_results_[???][???] for t in [f'param_{param}', 'mean_train_score', 'mean_test_score', 'std_test_score'])
    ax.plot(x, y, label='???')
    ax.plot(x, t, label='???')
    ax.fill_between(list(x), t - 2 * s, t + 2 * s, facecolor='orange', alpha=.2)
    ax.set(xlabel=param, ylabel='accuracy', title='Точность для ' + param)
    ax.legend()

**Вопросы:**

1. Опишите поведение графиков для каждого из параметров.
1. Изменился ли график максимального количества разбиения по сравнению с предыдущим вариантом? Почему?
1. Как связаны оптимальная сложность деревьев и количество распознающих деревьев?
1. За что отвечает параметр `max_samples`?

**Ответы:**

1. ...
1. ...

## Задание №4

Подготовьте датасет mnist через функцию `load_digits` для дальнейшей работы.
Преобразуйте каждое изображение в одномерный вектор нормированных признаков.
Разделите датасет на обучающую и тестовую выборки.
Размер тестовой выборки следует выбрать равным 20% от размера всей выборки.
Не забудьте указать `random_state` для повторяемости результатов.

In [ ]:
x, y = ???
x_train, x_test, y_train, y_test = ???

## Задание №5

Найдите наилучшие параметры для ансамблей, полученных баггингом, методом адаптивного бустинга, а также случайного леса с помощью `GridSearchCV`.
Выведите _accuracy_ для обучающей и тестовой выборок.
Параметры переборов:

* Макимальная глубина деревьев - от 1 до 20
* Количество деревьев - от 10 до 100
* `max_samples` и `max_features` - от 0 до 1

In [ ]:
tree_prototype = sk.tree.DecisionTreeClassifier(random_state=4)

params = {
    'BaggingClassifier': {
        'proto': sk.ensemble.???(??? random_state=4),
        'grid': {
            'base_estimator__max_depth': ??? .astype(int),
            'n_estimators': ??? .astype(int),
            'max_samples': ???}},
    'RandomForestClassifier': {
        'proto': sk.ensemble.???(random_state=4),
        'grid': {
            'max_depth': ???,
            'n_estimators': ???,
            'max_features': ???}},
    'AdaBoostClassifier': {
        'proto': sk.ensemble.???(??? random_state=4),
        'grid': {
            'base_estimator__max_depth' : ???,
            'n_estimators' : ???}}}

for (name, param) in params.items():
    gs = sk.model_selection.GridSearchCV(param['proto'], param_grid=param['grid'], n_jobs=-1)
    %time gs.fit(x_train, y_train)
    print(name, gs.best_params_)
    print('train', gs.score(x_train, y_train))
    print('test', gs.score(x_test, y_test))

**Вопросы:**

1. Какова оптимальная размерность подмножества случайно отбираемых признаков для случайного леса?
1. Какой метод позволил достичь наилучшего результата?
1. Что можно сказать о скорости обучения ансамблей?

**Ответы:**

1. ...
1. ...